In [1]:
import torch
from torch import nn, optim
from utils import load_data
import matplotlib.pyplot as plt
from utils import train_visualization, weight_initialization, shuffle
from torch.nn import functional as F



In [2]:
import time

In [3]:
train_input, train_target, train_classes, test_input, test_target, test_classes = load_data()

In [4]:
train_classes

tensor([[9, 3],
        [5, 4],
        [7, 4],
        ...,
        [1, 4],
        [3, 5],
        [1, 1]])

In [5]:
train_classes
concat = torch.zeros((1000,20))

for idx, (tgt1,tgt2)  in enumerate(train_classes): 
    
    concat[idx,tgt1] = 1
    concat[idx,10+tgt2] =1

In [6]:
concat[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0.])

In [7]:
target = torch.ones((train_classes.shape))


In [8]:
# Pas utile ... 
for i, (pair1,pair2) in enumerate(train_classes): 
    target[i] = torch.tensor([1,0]) if pair1 > pair2 else  torch.tensor([0,1])

In [164]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        
        ## Train pour mnist classificiation
        
        self.conv1 = nn.Conv2d(2, 32, kernel_size=5,padding=1) # 1 filter 32 channels , 5x5
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        
        
        
        
        
                              
        self.fc1 = nn.Linear(256, 200)
        self.drop = nn.Dropout(0.2)
        
        
        self.fc2 = nn.Linear(200, 1) # train_class cross + train_class 
        
        
        
        ## train pour reconnaitre 
        
        #self.fc3 = nn.Linear(20, 1) # decision layer 
        print(f'Parameters : {self.count_params()}')

    def forward(self, x):
        #print('input',x.shape)
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=2))
        #print('conv1',x.shape)


        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride = 1))
        
        #print('relu',x.shape)


        #print('conv2',x.shape)
        #print('ef',x.view(-1,256).shape)
        x = F.relu(self.fc1(x.view(-1,256)))
        x= self.drop(x)
        x = F.sigmoid(self.fc2(x))
       # print(x.shape)
        x= self.drop(x)
        
        #out1 = x
        
        #out2 = F.sigmoid(self.fc3(out1))

        return x.flatten()
        
    def count_params(self):
        return sum(p.numel() for p in self.parameters()if p.requires_grad)



In [170]:
def compute_accuracy(net, train,target):
    with torch.no_grad():
        out1 = net(train)
        #max_ = out1.argmax(1)

        for i, x in enumerate(out1):
            out1[i] = 1 if x > 0.5 else 0 
    return (out1 == target).sum().item()/len(target)




In [171]:
def train(net, train_input, train_target, train_classes, eta=1e-3, n_epochs=25, batch_size=100, verbose=True):
    criterion1 = nn.BCELoss()
    criterion2 = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(net.parameters(), lr=eta, weight_decay=1e-5)
    
    losses, accuracies = [], []
    
    for e in range(n_epochs):
        sum_loss = 0
        loss_auxs=0
        net.train()
        for b in range(0, train_input.size(0), batch_size):
            trainX = train_input.narrow(0, b, batch_size)
            trainC = train_target.narrow(0, b, batch_size)
            trainT = concat.narrow(0, b, batch_size)
            

            final = net(trainX)
            
            #print(out1.shape,trainC.shape)
            
            
            #loss_aux = criterion2(aux,trainT.float())
            
           # print(trainX.shape)
            
            loss_final = criterion1(final,trainC.float())
            
            
            loss =   loss_final
            
            
            sum_loss += loss.item()
           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        net.eval()
        with torch.no_grad():
            acc = compute_accuracy(net, train_input, train_target)
        losses.append(sum_loss)
        accuracies.append(acc)
        if verbose:
            print('Epoch %d/%d, Loss: %.3f, Auxiliary: %.3f,  Accuracy: %.3f' % (e+1, n_epochs, sum_loss,0,acc))
    
    return losses, accuracies

def trial(net, train_input, train_classes, train_target, test_input, test_target, n_trials=10):
    all_losses = []
    all_train_accuracies = []
    all_test_accuracies = []
    for i in range(n_trials):
        print(f'Trial {i+1}/{n_trials}...')
        # Shuffle data
        train_input, train_target, train_classes = shuffle(train_input, train_target, train_classes)
        
        # Reset weights
        net.train()
        net.apply(weight_initialization)
        
        # Train
        train_loss, train_acc = train(net, train_input, train_target, train_classes, verbose=False)
        
        # Collect data
        all_losses.append(train_loss)
        all_train_accuracies.append(train_acc)
        net.eval()
        with torch.no_grad():
            test_acc = compute_accuracy(net, test_input, test_target)
            all_test_accuracies.append(test_acc)
        
        print('Loss: %.3f, Train accuracy: %.3f, Test_accuracy: %.3f' % (train_loss[-1], train_acc[-1], test_acc))
    
    return torch.FloatTensor(all_losses), torch.FloatTensor(all_train_accuracies), torch.FloatTensor(all_test_accuracies)

In [172]:
net = Net()

Parameters : 71729


In [173]:
start = time.time()
train(net, train_input, train_target, train_classes, batch_size=100)
print(time.time()-start)

Epoch 1/25, Loss: 6.639, Auxiliary: 0.000,  Accuracy: 0.717
Epoch 2/25, Loss: 5.718, Auxiliary: 0.000,  Accuracy: 0.743
Epoch 3/25, Loss: 5.022, Auxiliary: 0.000,  Accuracy: 0.769
Epoch 4/25, Loss: 4.511, Auxiliary: 0.000,  Accuracy: 0.809
Epoch 5/25, Loss: 4.083, Auxiliary: 0.000,  Accuracy: 0.822
Epoch 6/25, Loss: 3.705, Auxiliary: 0.000,  Accuracy: 0.838
Epoch 7/25, Loss: 3.387, Auxiliary: 0.000,  Accuracy: 0.852
Epoch 8/25, Loss: 3.097, Auxiliary: 0.000,  Accuracy: 0.883
Epoch 9/25, Loss: 2.836, Auxiliary: 0.000,  Accuracy: 0.898
Epoch 10/25, Loss: 2.644, Auxiliary: 0.000,  Accuracy: 0.883
Epoch 11/25, Loss: 2.532, Auxiliary: 0.000,  Accuracy: 0.926
Epoch 12/25, Loss: 2.176, Auxiliary: 0.000,  Accuracy: 0.906
Epoch 13/25, Loss: 1.875, Auxiliary: 0.000,  Accuracy: 0.947
Epoch 14/25, Loss: 1.597, Auxiliary: 0.000,  Accuracy: 0.964
Epoch 15/25, Loss: 1.408, Auxiliary: 0.000,  Accuracy: 0.957
Epoch 16/25, Loss: 1.299, Auxiliary: 0.000,  Accuracy: 0.959
Epoch 17/25, Loss: 1.248, Auxilia

In [174]:
start = time.time()
tr_loss, tr_acc, te_acc = trial(net, train_input, train_classes, train_target, test_input, test_target)


print(time.time()-start)

Trial 1/10...
Loss: 0.078, Train accuracy: 1.000, Test_accuracy: 0.807
Trial 2/10...
Loss: 0.055, Train accuracy: 1.000, Test_accuracy: 0.817
Trial 3/10...
Loss: 0.031, Train accuracy: 1.000, Test_accuracy: 0.814
Trial 4/10...
Loss: 1.371, Train accuracy: 0.997, Test_accuracy: 0.800
Trial 5/10...
Loss: 0.632, Train accuracy: 0.985, Test_accuracy: 0.802
Trial 6/10...
Loss: 0.201, Train accuracy: 1.000, Test_accuracy: 0.820
Trial 7/10...
Loss: 1.935, Train accuracy: 1.000, Test_accuracy: 0.820
Trial 8/10...
Loss: 0.001, Train accuracy: 1.000, Test_accuracy: 0.825
Trial 9/10...
Loss: 0.001, Train accuracy: 1.000, Test_accuracy: 0.822
Trial 10/10...
Loss: 0.001, Train accuracy: 1.000, Test_accuracy: 0.814
61.48161697387695


In [175]:

model, criterion = Net(), nn.MSELoss()
eta, mini_batch_size = 1e-1, 100

def train_model(model, train_input, train_target, mini_batch_size): 
    loss_ = []
    out = torch.empty((25,1000,10))
    for e in range(0, 25):
        sum_loss = 0
    # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            out[e,b:b+100] = output
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad
        loss_.append(sum_loss)
    return loss_, out , model 

    
    

losses,_,model_ = train_model(model, train_input, train_target, mini_batch_size)

Parameters : 71729


RuntimeError: The expanded size of the tensor (10) must match the existing size (100) at non-singleton dimension 1.  Target sizes: [100, 10].  Tensor sizes: [100]

In [ ]:
x= torch.ones((100,64,1,1))

In [47]:
x.reshape(100,256).shape

RuntimeError: shape '[100, 256]' is invalid for input of size 100

In [46]:
x= torch.ones([100])

In [32]:
x1=torch.ones([100])

In [46]:
(x==x1).sum(0).item()/len(x)


1.0

In [76]:
train_target.shape

torch.Size([1000])